## Non-normal Naive Bayes Classifier

In this section, we will implement a non-normal Naive Bayes classifier. Unlike the standard Naive Bayes classifier, which assumes that the features follow a normal distribution, the non-normal Naive Bayes classifier makes no such assumptions.

Having explored the dataset beforehand, we know that the features do not follow a normal distribution. However, most of them seem to follow parametric distributions such as exponential, Poisson, or multinomial distributions. For that reason, three different non-normal Naive Bayes classifiers will be implemented: In the first one, we will use histogram-based probability density estimation for each continuous feature. In the second one, we will use kernel density estimation (KDE) for each continuous feature. Finally, in the third one, we will attempt to discover the underlying parametric distribution of each feature and use that information to build the classifier.

Let us start by loading the data, removing unnecessary columns, preprocessing the rest, and splitting it into training and test sets. This part is similar to what we did in the previous sections. However, because the density estimation methods we will use in this section are tunable (e.g., number of bins in histograms, bandwidth in KDE), we will also implement a K-fold cross-validation procedure to find the best hyperparameters for each method.

In [5]:
import pandas as pd


df = pd.read_csv("../data/raw/higgs-challenge.csv.gz", compression="gzip")
df = df.drop(columns=["EventId", "KaggleSet", "KaggleWeight"])
df.head()

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label
0,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,197.760,...,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497,0.000814,s
1,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,125.157,...,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226,0.681042,b
2,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,197.814,...,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251,0.715742,b
3,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,75.968,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,-0.000,1.660654,b
4,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,57.983,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000,1.904263,b


In [6]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["Label", "Weight"])
weight = df["Weight"]  # Only used to compute the AMS metric
y = df["Label"]
# Stratify to maintain class distribution
X_train, X_test, weight_train, weight_test, y_train, y_test = train_test_split(
    X, weight, y, test_size=0.2, random_state=42, stratify=y
)
X_train.shape, X_test.shape, weight_train.shape, weight_test.shape, y_train.shape, y_test.shape

((654590, 30), (163648, 30), (654590,), (163648,), (654590,), (163648,))

### Non-normal Naive Bayes Classifier with Histogram-based Estimation

Let us start by implementing a trivial non-normal Naive Bayes classifier that uses histogram-based probability density estimation for each continuous feature. The idea is to create histograms for each feature conditioned on the class labels and use these histograms to estimate the likelihoods during prediction.

In [ ]:
from typing import Optional
from sklearn.naive_bayes import _BaseNB
import numpy as np
from sklearn.utils.validation import validate_data
from numpy.typing import NDArray
from typing import TypeVar
from typing import List

YType = TypeVar("YType", bound=np.generic)


class HistogramNB(_BaseNB):
    """
    Histogram Naive Bayes (HistogramNB).

    Args:
        bin_count (Optional[int]): Number of bins to use for histogram estimation.
            If None, each histogram will use a number of bins equal to the square
            root of the number of samples.
    """

    _priors: NDArray[np.float64]
    _class_counts: NDArray[np.float64]
    _class_histograms: List[NDArray[np.float64]]
    _bin_edges: List[NDArray[np.float64]]
    classes_: NDArray[YType]

    def __init__(self, bin_count: Optional[int] = None):
        self.bin_count = bin_count
        super().__init__()

    def fit(self, X: NDArray[np.float64], y: NDArray[YType]):
        """Fit Histogram Naive Bayes according to X, y.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training vectors, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        y : array-like of shape (n_samples,)
            Target values.

        sample_weight : array-like of shape (n_samples,), default=None
            Weights applied to individual samples (1. for unweighted).

            .. versionadded:: 0.17
               Histogram Naive Bayes supports fitting with *sample_weight*.

        Returns
        -------
        self : object
            Returns the instance itself.
        """
        X, y = validate_data(self, X, y, reset=True)
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)
        self._class_counts = np.zeros(n_classes, dtype=np.float64)
        self._class_histograms = [None] * n_classes
        self._bin_edges = [None] * n_classes

        for i, y_i in enumerate(self.classes_):
            X_i = X[y == y_i, :]
            N_i = X_i.shape[0]

            # Placeholder: Update the histograms
            if self.bin_count is None:
                bin_count = int(np.sqrt(N_i))
            else:
                bin_count = self.bin_count
            self._class_histograms[i] = np.empty((X.shape[1], bin_count))
            self._bin_edges[i] = np.empty((X.shape[1], bin_count + 1))
            for feature_idx in range(X.shape[1]):
                self._class_histograms[i][feature_idx], self._bin_edges[i][feature_idx] = np.histogram(
                    X_i[:, feature_idx], bins=bin_count, density=True
                )

            self._class_counts[i] += N_i

        self._priors = self._class_counts / self._class_counts.sum()

        return self

    def _joint_log_likelihood(self, X: NDArray[np.float64]) -> NDArray[np.float64]:
        # TODO: Check this implementation again... (It was vibe coded at 2 AM)
        joint_log_likelihood = []
        for i, y_i in enumerate(self.classes_):
            log_likelihood = np.log(self._priors[i]) * np.ones(X.shape[0])
            for feature_idx in range(X.shape[1]):
                bin_edges = self._bin_edges[i][feature_idx]
                hist = self._class_histograms[i][feature_idx]
                bin_indices = np.digitize(X[:, feature_idx], bin_edges) - 1
                bin_indices = np.clip(bin_indices, 0, len(hist) - 1)
                likelihoods = hist[bin_indices] * np.diff(bin_edges)[bin_indices]
                likelihoods[likelihoods == 0] = 1e-9
                log_likelihood += np.log(likelihoods)
            joint_log_likelihood.append(log_likelihood)
        return np.array(joint_log_likelihood).T

    def _check_X(self, X: NDArray[np.float64]) -> NDArray[np.float64]:
        """Validate X against the training data."""
        X = validate_data(self, X, reset=False)
        return X

This implementation has a `n_bins` hyperparameter that specifies the number of bins to use for the histograms. We will use K-fold cross-validation to find the best value for this hyperparameter. But first, let us perform a quick training run with `bin_count` set to 10, and all the learning data to see how well this classifier performs without any hyperparameter tuning.

In [ ]:
model = HistogramNB(bin_count=10)
model.fit(X_train.to_numpy(), y_train.to_numpy())

y_pred = model.predict(X_test.to_numpy())
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test.to_numpy(), y_pred)
report = classification_report(y_test.to_numpy(), y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(report)

Accuracy: 0.6522
              precision    recall  f1-score   support

           b       0.73      0.74      0.74    107736
           s       0.49      0.48      0.49     55912

    accuracy                           0.65    163648
   macro avg       0.61      0.61      0.61    163648
weighted avg       0.65      0.65      0.65    163648



### Non-normal Naive Bayes Classifier with Kernel Density Estimation

Next, we will implement a non-normal Naive Bayes classifier that uses kernel density estimation (KDE) for each continuous feature. KDE is a non-parametric way to estimate the probability density function of a random variable famously used for Naive Bayes classifiers (Trevor Hastie et al.). We will use Gaussian kernels for this purpose.

#### Definition of KDE-based Naive Bayes Classifier

Let $(x_1, ..., x_n)$ be a set of independent and identically distributed samples drawn from some unknown probability density function $f$. The kernel density estimate $\hat{f}(x)$ of $f$ at a point $x$ is given by:

$$
\hat{f}(x) = \frac{1}{n} \sum_{i=1}^{n} K_h(x - x_i)
$$

where $K_h$ is the kernel function with bandwidth $h$, a smoothing parameter related to the variance of the estimator. For a Gaussian kernel, this is defined as:

$$
K_h(u) = \frac{1}{\sqrt{2\pi h^2}} \exp\left(-\frac{u^2}{2h^2}\right)
$$



## References

- Hastie, T., Tibshirani, R., & Friedman, J. (2001). The Elements of Statistical Learning: Data Mining, Inference, and Prediction: with 200 full-color illustrations. New York: Springer.